In [1]:
!pip list

Package           Version
----------------- --------
asttokens         2.0.7
backcall          0.2.0
cramjam           2.5.0
debugpy           1.6.2
decorator         5.1.1
entrypoints       0.4
executing         0.9.1
fastparquet       0.8.1
fsspec            2022.7.1
funcy             1.17
future            0.18.2
gensim            3.8.3
ipykernel         6.15.1
ipython           8.4.0
jedi              0.18.1
Jinja2            3.1.2
joblib            1.1.0
jupyter-client    7.3.4
jupyter-core      4.11.1
MarkupSafe        2.1.1
matplotlib-inline 0.1.3
nest-asyncio      1.5.5
numexpr           2.8.3
numpy             1.23.1
packaging         21.3
pandas            1.4.3
parso             0.8.3
pexpect           4.8.0
pickleshare       0.7.5
pip               22.2.2
prompt-toolkit    3.0.30
psutil            5.9.1
ptyprocess        0.7.0
pure-eval         0.2.2
Pygments          2.12.0
pyLDAvis          3.3.1
pyparsing         3.0.9
python-dateutil   2.8.2
pytz              2022.2
pyz

In [2]:
### aqui é para os testes 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.datasets import fetch_20newsgroups

### Isso é para a classe do professor Thiago
from sklearn.base import BaseEstimator,TransformerMixin
from gensim.matutils import Sparse2Corpus, corpus2dense
from gensim.models.wrappers import LdaMallet
from gensim.corpora.dictionary import Dictionary
import numpy as np
from ldamallet import LdaMalletHandler
import pandas as pd

In [3]:
path = '/home/anasofia/Documents/Projetos/AILab/ALEI-1a/dados/4a_vara/98processos4vara.csv'

docs = pd.read_csv(path)

In [4]:
docs['Texto'].head

<bound method NDFrame.head of 0       \n \nCuida-se de ação de rito comum ordinári...
1       \n \n \n  \nTrata-se de procedimento comum a...
2      (156) \nPOLO ATIVO: AURISTELA MACIEL LINS \nR...
3       \n \nTrata-se de Procedimento Comum Cível pr...
4      \n  \n  \n \nA parte autora requereu a desist...
                            ...                        
93     \n \n \n  \nCuida-se de procedimento comum cí...
94     CONTRA A FAZENDA PÚBLICA (12078) \nPOLO ATIVO...
95     (156) \nPOLO ATIVO: AGRICHEM DO BRASIL S.A. \...
96      \n \n \n  \nTrata-se de ação ordinária ajuiz...
97     CONTRA A FAZENDA PÚBLICA (12078) \nPOLO ATIVO...
Name: Texto, Length: 98, dtype: object>

In [5]:
vect = CountVectorizer(strip_accents = 'unicode',
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 0.7, 
                                min_df = 0.1)

In [6]:
X = vect.fit_transform(docs['Texto'])

In [7]:
path_to_mallet_binary = "/home/anasofia/Mallet/bin/mallet"

In [8]:
ldamodel = LdaMalletHandler(n_components=20,
                            mallet_path=path_to_mallet_binary,
                            iterations=100,
                            vectorizer=vect)

In [9]:
a, b = ldamodel.vect2gensim(vect, X)

In [10]:
model = LdaMallet(path_to_mallet_binary,
          iterations=100,
          corpus=a,
          num_topics=30,
          id2word=b)

Mallet LDA: 30 topics, 5 topic bits, 11111 topic mask
Data loaded.
max tokens: 1097
total tokens: 35903
<10> LL/token: -7.78796
<20> LL/token: -7.29336
<30> LL/token: -7.08863
<40> LL/token: -6.99599

0	1.66667	juros incidencia pelo mora min quanto selic tributario tribunal ctn autos quando forma contra taxa outras pedidos exercicio desta citada 
1	1.66667	contribuicoes recurso julgamento pelo acordao julgado terceiros relacao merito stf termos qual ajuizada seja juridico trf dias efeito presente publicacao 
2	1.66667	calculo base repercussao geral exclusao extraordinario recurso supremo tese stf min razao servicos sistematica julgamento dje seguinte diante tema presente 
3	1.66667	montante icms operacao compensacao autora ate faturamento mes apenas correspondente tambem regime todo nem contabil parte recolhido juridica principio contribuicoes 
4	1.66667	declaracao trata custas honorarios autora termos procedimento acao tenham inicialmente provas ante podem pronto encontra muito comum 

In [11]:
path_to_model = model.fstate()
path_to_model

'/tmp/615b09_state.mallet.gz'

In [12]:
model.fdoctopics()

'/tmp/615b09_doctopics.txt'

In [13]:
ldamodel.model = model

In [14]:
!pip install pyldavis

In [28]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
from gensim.matutils import Sparse2Corpus
from gensim.corpora.dictionary import Dictionary
from gensim.models.wrappers import LdaMallet
from gensim.models.ldamodel import LdaModel

In [29]:
def convertldaMalletToldaGen(mallet_model):
    model_gensim = LdaModel(
        id2word=mallet_model.id2word, num_topics=mallet_model.num_topics,
        alpha=mallet_model.alpha) 
    model_gensim.state.sstats[...] = mallet_model.wordtopics
    model_gensim.sync_state()
    return model_gensim

In [30]:
corpus_vect_gensim = Sparse2Corpus(X, documents_columns=False)
dictionary = Dictionary.from_corpus(corpus_vect_gensim,
        id2word=dict((id, word) for word, id in vect.vocabulary_.items()))
ldagensim = convertldaMalletToldaGen(model)

In [32]:
pyLDAvis.enable_notebook()
gensimvis.prepare(ldagensim, corpus_vect_gensim, dictionary)

/home/anasofia/Documents/Projetos/AILab/ALEI-1a/LdaMallet-temp/env/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
12     0.066889 -0.030288       1        1  6.110611
9      0.111266 -0.046246       2        1  6.089674
22     0.188454 -0.023547       3        1  4.511064
7      0.187770 -0.028468       4        1  4.487501
14     0.093208  0.070631       5        1  4.407015
8      0.117997  0.067150       6        1  4.367698
19    -0.156924  0.089168       7        1  3.887777
3     -0.009503 -0.094370       8        1  3.851145
20    -0.095002  0.011371       9        1  3.683001
24    -0.064800  0.108917      10        1  3.580579
5     -0.102207 -0.003393      11        1  3.425754
4     -0.033341  0.136915      12        1  3.401315
16    -0.043819  0.209384      13        1  3.314812
26    -0.206420  0.092996      14        1  3.268644
17    -0.050475 -0.171379      15        1  3.266186
27    -0.026781 -0.042752      16        1  3.240363
6      0.080387 -0.086764      17        1  3.215175
1      0.031294  0.155732      18        1  2.905674
10     0.034495 -0.160405      19        1  2.833306
25    -0.211470 -0.029368      20        1  2.823217
28    -0.094854 -0.103627      21        1  2.823065
2      0.220790  0.045704      22        1  2.781375
23    -0.011720 -0.125615      23        1  2.759133
15    -0.179209 -0.193547      24        1  2.701757
21     0.063174 -0.104194      25        1  2.503978
11    -0.004930  0.186472      26        1  2.486812
29     0.048742 -0.081077      27        1  2.385385
18    -0.081811  0.062169      28        1  2.322553
0      0.088668  0.099151      29        1  1.936425
13     0.040134 -0.010721      30        1  0.629004, topic_info=             Term        Freq       Total Category  logprob  loglift
393           lei  530.000000  530.000000  Default  30.0000  30.0000
141       calculo  266.000000  266.000000  Default  29.0000  29.0000
142  contribuicao  373.000000  373.000000  Default  28.0000  28.0000
162          icms  210.000000  210.000000  Default  27.0000  27.0000
140          base  265.000000  265.000000  Default  26.0000  26.0000
..            ...         ...         ...      ...      ...      ...
148           uma    0.742985  108.890029  Topic30  -5.7197   0.0814
305      montante    0.742985  113.374645  Topic30  -5.7197   0.0410
0           cuida    0.023967   19.489130  Topic30  -9.1537  -1.6321
1            acao    0.023967  107.215479  Topic30  -9.1537  -3.3371
2            rito    0.023967   22.490278  Topic30  -9.1537  -1.7754

[1227 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
266       8  0.051842     abrir
266      15  0.881308     abrir
1        12  0.111924      acao
1        16  0.018654      acao
1        17  0.373080      acao
...     ...       ...       ...
725      19  0.077610  violacao
725      24  0.388048  violacao
267      15  0.760309     vista
267      20  0.200081     vista
267      22  0.020008     vista

[2073 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[13, 10, 23, 8, 15, 9, 20, 4, 21, 25, 6, 5, 17, 27, 18, 28, 7, 2, 11, 26, 29, 3, 24, 16, 22, 12, 30, 19, 1, 14])

/home/anasofia/Documents/Projetos/AILab/ALEI-1a/LdaMallet-temp/env/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/anasofia/Documents/Projetos/AILab/ALEI-1a/LdaMallet-temp/env/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/anasofia/Documents/Projetos/AILab/ALEI-1a/LdaMallet-temp/env/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/anasofia/Documents/Projetos/AILab/ALEI-1a/LdaMallet-temp/env/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of impo

In [ ]:
from dfrbrowser import dfrBrowserConverter

In [ ]:
data = docs['Texto']
data.to_csv('meta.tsv', sep='\t', index=False, header=True)

In [ ]:
conv = dfrBrowserConverter(ldamodel, path='dfr-browser/data')
conv.generate_files('aaaaaa', 'BBBBBBBBJDSAFKLDSJAFKJLS', data=data)